## County Choropleth Maps

Filling county data for **county level choropleth map** is a bit tricky. 

`geom_map()` **always** expects the **same** column names in a map data frame as we have seen (either (long,lat,region) or (x,y,id)), and **always** links the data column specified by the `map_id` aes to the `region` or `id` column in the map data frame.

So, you can draw and fill states just fine as you have seen in the practice; but for counties, you have to make a change in the map data frame.

**See the example below:** 


In [ ]:
library(ggplot2)

# county level map data frame for south carolina

sc <- map_data('county', 'south carolina')
head(sc)



Let's create some made-up data frame to have some values linked to the county data.

**We need to make sure that the county names are NOT categories but strings so that they can verbatimly match the county names in the map data frame.** 


In [ ]:
dsf <- data.frame(county=as.character(unique(sc$subregion)), pop = seq(1,46),  stringsAsFactors = FALSE)
head(dsf)
str(dsf)

**Here is the trick:** geom_map ALWAYS links to the `region` column; so we can do the following to create a region column with county names:

In [ ]:
sc$region = sc$subregion
head(sc)

Now, the `region` column contains counties; we can link our data to counties.

In [ ]:
# use dsf as the data frame and fill the regions with pop
ggplot(data=dsf, aes(fill=pop)) + 

# link "county" column in dsf to "region" column in sc 
geom_map(map=sc, aes(map_id=county), color="white", size=0.1) +  

# you should have this for the map to appear correctly
expand_limits(x = sc$long, y = sc$lat) + 

coord_map("polyconic") +
theme_void()

### Let's see how to draw just the boundaries



In [ ]:
# get the ORIGINAL map data frame again 

sc2 <- map_data('county', 'south carolina')
head(sc2)

In [ ]:
ggplot() + 
# use sc2 as BOTH data frame and map 

geom_map(data=sc2, map=sc2, aes(map_id=region), fill=NA, color="black") + 

expand_limits(x=sc2$long, y=sc2$lat) +
coord_map("polyconic") + 
theme_void()

**So why did it work?** 

We did not assign `subregion` to `region` in `sc2`, and it still worked, because we are **not** filling regions with a data frame attribute. We use the map data frame **both** as map and data to draw just the boundaries, so we don't need to link the county names to anything. 


**This example does both as two layers:**


In [ ]:
ggplot() + 

# use dsf as data frame and fill the "region"s with pop
geom_map(data=dsf, map=sc, aes(map_id=county, fill=pop)) +  # draw "region" and link "county" in dsf to "region" in sc map 

geom_map(data=sc, map=sc, aes(map_id=region), fill=NA, color="lightblue") + # link "region" in sc data to "region" in sc map 

expand_limits(x = sc$long, y = sc$lat) + # you should ALWAYS have this for the map to appear correctly. 

coord_map("polyconic") +

theme_void()

Of course, you could have the same visual result with one geom_map, too, but the example above shows how you can manipulate map data frames as separate layers if you want. 

**The following has the same visual output as above:** 

In [ ]:
ggplot() +

geom_map(data=dsf, map=sc, aes(map_id=county, fill=pop), color="lightblue") +  # draw "region" and link "county" in dsf to "region" in sc map

expand_limits(x = sc$long, y = sc$lat) + # you should ALWAYS have this for the map to appear correctly.

coord_map("polyconic") +

theme_void()

---

We can also add text labels on the map. Below, we compute the mean coordinates for each county to create a data frame that contains county names and (long,lat) pairs that approximately represent the centers of counties. Then we can use `geom_text` or `geom_text_repel` to add names on the map.  

In [ ]:
library(dplyr)
name_data <- sc %>%
  group_by(region) %>%
  summarise(long = mean(long), lat = mean(lat))

head(name_data)

In [ ]:
library(ggrepel)

ggplot() + 

geom_map(data=dsf, map=sc, aes(map_id=county, fill=pop), color="white", size=0.1, alpha=0.5) +  

geom_text_repel(data=name_data, aes(x=long, y=lat, label=region)) + 

expand_limits(x = sc$long, y = sc$lat) + 
coord_map("polyconic") +
theme_void()

---

We can overlay this choropleth map on top of other layers like in the following example. 


You'll remember part of this code from practices:

In [ ]:
state <- map_data("state")
county <- map_data("county")
usa <- map_data("usa")

gg <- ggplot() + 
 geom_map(data=county, map=county, aes(map_id=region), color="grey", fill=NA, size=0.1) +
 geom_map(data=state, map=state, aes(map_id=region), color="red", fill=NA, size=0.3) +
 geom_map(data=usa, map=usa, aes(map_id=region), color="blue", fill=NA, size=1) +

 geom_map(data=dsf, map=sc, aes(map_id=county, fill=pop), color="lightblue", size=0.1) + 
 
 coord_map("albers", lat0=30, lat1=40) + 
 expand_limits(x = usa$long, y = usa$lat) +
 theme_void()
gg

**We can change the limits to focus on the South Carolina.** 

Also, look how the **order** of geom_maps dictate the drawing order of layers. 

In [ ]:
state <- map_data("state")
county <- map_data("county")
usa <- map_data("usa")

gg <- ggplot() + 
 geom_map(data=county, map=county, aes(map_id=region), color="grey", fill=NA, size=0.1) +
 geom_map(data=state, map=state, aes(map_id=region), color="red", fill=NA, size=0.3) +

 geom_map(data=dsf, map=sc, aes(map_id=county, fill=pop), color="lightblue", size=0.1) + 

 geom_map(data=usa, map=usa, aes(map_id=region), color="blue", fill=NA, size=1) +
 
 coord_map("albers", lat0=30, lat1=40) + 
 expand_limits(x = sc$long, y = sc$lat) + 
 theme_void()
gg

## YOUR TURN: 
**Repeat the same visualization from the cell above for Missouri.** 

Make sure to **expand** the limits of the map to show **the neighbor states in their entirety** (for example, entire state of Kansas should be visible, etc.).  

You can make up the data frame for fill values. 

(Hint: use latlong.net to find coordinates for xlim() and ylim().) 

In [ ]:
mo <- map_data('county', 'missouri')

# make up some data for pop
dsf <- data.frame(county=as.character(unique(mo$subregion)), pop = seq(1,115),  stringsAsFactors = FALSE)
head(dsf)
str(dsf)

# do the trick so that region column in the map data has county names 
mo$region = mo$subregion

In [ ]:
gg <- ggplot() + 

 geom_map(data=county, map=county, aes(map_id=region), color="grey", fill=NA, size=0.1) +
 geom_map(data=state, map=state, aes(map_id=region), color="red", fill=NA, size=0.3) +

 geom_map(map=mo, data=dsf, aes(map_id=county, fill=pop), color="lightblue", size=0.1) + 

 geom_map(data=usa, map=usa, aes(map_id=region), color="blue", fill=NA, size=1) +

 
 coord_map("albers", lat0=30, lat1=40) +

 # these are the coordinates you can find from latlong.net to make sure neighbor states are contained in the map 
 ylim(c(33, 44)) + xlim(c(-102, -82)) + 
 theme_void()
gg